In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tree-features/test.pickle
/kaggle/input/tree-features/val.pickle
/kaggle/input/tree-features/train.pickle
/kaggle/input/total-data/val_data.csv
/kaggle/input/total-data/train_data.csv
/kaggle/input/total-data/test_data.csv


In [2]:
# All external libraries used throughout the notbook are listed here

from tqdm import tqdm
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier as logreg_w_sgd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import pickle
import numpy as np
import pandas as pd
import nltk

In [3]:
ROOT = "/kaggle/input/"

SAVED_MODEL_PATH = ROOT + "models/"

TREE_FEAT_PATH = ROOT + "tree-features/"

DATA_PATH = ROOT + "total-data/"

In [4]:
# train_data.to_csv('train_data.csv', index=False)
# val_data.to_csv('val_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)
#with open(TREE_FEAT_PATH + "train.pickle", "rb") as f_train:

train_data = pd.DataFrame(pd.read_pickle(TREE_FEAT_PATH + "train.pickle"))
val_data = pd.DataFrame(pd.read_pickle(TREE_FEAT_PATH + "val.pickle"))
test_data = pd.DataFrame(pd.read_pickle(TREE_FEAT_PATH + "test.pickle"))

# Combine both Training and Validation data.
train_val_data = pd.concat([train_data, val_data], axis = 0, ignore_index=True)

In [5]:
# Import labels...
train_labels = pd.read_csv(DATA_PATH + "train_data.csv", index_col=0).is_duplicate.values
val_labels = pd.read_csv(DATA_PATH + "val_data.csv", index_col=0).is_duplicate.values
test_labels = pd.read_csv(DATA_PATH + "test_data.csv", index_col=0).is_duplicate.values

# Combine train and val labels.
train_val_labels = np.concatenate((train_labels, val_labels))

# Apply GridSearch with 5-Fold CV.

## Gradient Boosting.

In [6]:
# Apply gridSearch CV.
params_gb = {"loss":["deviance", "exponential"],
             "n_estimators":[20, 50, 75], 
             "min_samples_split":[128, 200, 256]}
             # "criterion": ["squared_error", "friedman_mse"]
             # "min_samples_split":[16, 32, 64] = 128 is opt,
             # "subsample":[0.4, 0.8, 1.0],# "learning_rate": [0.05, 0.1, 0.8], <-------------------------------
gb_ = GradientBoostingClassifier(max_features = "log2", max_depth = 25, min_samples_split = 32)
# Cv -splitter
cv_splitter = StratifiedKFold(n_splits=5, shuffle=False)
gb_cv = GridSearchCV(gb_, params_gb, scoring = ["accuracy", "f1"], n_jobs = -1, cv = cv_splitter, refit="accuracy")
gb_cv.fit(train_val_data, train_val_labels)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingClassifier(max_depth=25,
                                                  max_features='log2',
                                                  min_samples_split=32),
             n_jobs=-1,
             param_grid={'loss': ['deviance', 'exponential'],
                         'min_samples_split': [128, 200, 256],
                         'n_estimators': [20, 50, 75]},
             refit='accuracy', scoring=['accuracy', 'f1'])

In [7]:
# Show the Results...
gb_res = pd.DataFrame.from_dict(gb_cv.cv_results_)
gb_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_loss,param_min_samples_split,param_n_estimators,params,split0_test_accuracy,split1_test_accuracy,...,std_test_accuracy,rank_test_accuracy,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,55.791932,0.602557,0.747229,0.031692,deviance,128,20,"{'loss': 'deviance', 'min_samples_split': 128,...",0.727797,0.729182,...,0.000693,18,0.624025,0.629242,0.626104,0.626440,0.625250,0.626212,0.001729,18
1,120.887408,1.219311,1.305888,0.068758,deviance,128,50,"{'loss': 'deviance', 'min_samples_split': 128,...",0.727687,0.730061,...,0.001618,14,0.637708,0.642810,0.638331,0.643497,0.640256,0.640521,0.002318,11
2,166.413243,2.346910,1.693135,0.046199,deviance,128,75,"{'loss': 'deviance', 'min_samples_split': 128,...",0.726739,0.729800,...,0.001717,17,0.636918,0.643534,0.637328,0.642071,0.640553,0.640081,0.002596,12
3,41.403875,0.440530,0.760083,0.066597,deviance,200,20,"{'loss': 'deviance', 'min_samples_split': 200,...",0.727523,0.730308,...,0.001233,13,0.623245,0.630256,0.627339,0.628446,0.626617,0.627181,0.002319,16
4,96.643226,1.611613,1.215221,0.020182,deviance,200,50,"{'loss': 'deviance', 'min_samples_split': 200,...",0.729982,0.732177,...,0.001133,7,0.641633,0.647521,0.640825,0.645082,0.643358,0.643684,0.002415,7
5,135.697323,1.097747,1.577229,0.047317,deviance,200,75,"{'loss': 'deviance', 'min_samples_split': 200,...",0.729845,0.731133,...,0.001063,8,0.642064,0.645839,0.641723,0.645177,0.643269,0.643614,0.001642,8
6,37.094729,0.414630,0.682775,0.004702,deviance,256,20,"{'loss': 'deviance', 'min_samples_split': 256,...",0.729790,0.731133,...,0.000971,9,0.627293,0.631414,0.626645,0.629090,0.628472,0.628583,0.001656,13
7,86.682292,0.964423,1.193612,0.039640,deviance,256,50,"{'loss': 'deviance', 'min_samples_split': 256,...",0.729707,0.731957,...,0.001390,4,0.642714,0.646156,0.642216,0.646300,0.644318,0.644341,0.001691,5
8,125.465447,1.548001,1.482307,0.013492,deviance,256,75,"{'loss': 'deviance', 'min_samples_split': 256,...",0.730271,0.732383,...,0.001119,2,0.643129,0.647530,0.644356,0.647484,0.645245,0.645549,0.001735,1
9,50.428197,1.126804,0.746759,0.015597,exponential,128,20,"{'loss': 'exponential', 'min_samples_split': 1...",0.728402,0.729456,...,0.000984,16,0.624418,0.628486,0.624945,0.627929,0.625896,0.626335,0.001610,17


In [8]:
# Show the Best params, and Best Score...
print(f"CV for Gradient Boosting Classifier:\nBest Parameters: {gb_cv.best_params_}\nBest Scores: {gb_cv.best_score_}")

CV for Gradient Boosting Classifier:
Best Parameters: {'loss': 'exponential', 'min_samples_split': 256, 'n_estimators': 50}
Best Scores: 0.7324857648992078


In [9]:
# Print the best F1 and Accuracy score.
print("F1 score: ", np.max(gb_res["mean_test_f1"]), "Accuracy Score", np.max(gb_res["mean_test_accuracy"]))

F1 score:  0.6455486305780218 Accuracy Score 0.7324857648992078


In [10]:
# Now train the model with optimal hyper-parameters and obtain the result on test data-set.
gb_optimal = GradientBoostingClassifier(max_features = "log2",
                                        max_depth = 25,
                                        n_estimators = gb_cv.best_params_["n_estimators"],
                                        loss = gb_cv.best_params_["loss"],
                                        min_samples_split = gb_cv.best_params_["min_samples_split"])
# Train the DT.
gb_optimal = gb_optimal.fit(train_val_data, train_val_labels)
# Predict on vals.
pred = gb_optimal.predict(test_data)
# Accuracy and F1 score.
f1_gb_optimal = f1_score(test_labels, pred)
acc_gb_optimal = accuracy_score(test_labels, pred)
# Print the scores.
print("Accuracy-Score for Gradient Boosting Classifier with Optimal Params = %.3f"%acc_gb_optimal)
print("F1-Score for Gradient Boosting Classifier with Optimal Params = %.3f"%f1_gb_optimal)

Accuracy-Score for Gradient Boosting Classifier with Optimal Params = 0.737
F1-Score for Gradient Boosting Classifier with Optimal Params = 0.651


**save the model to disk**
pickle.dump(gb_cv, open(ROOT + 'GB_CV_model.sav', 'wb'))